<center> ALGORITMOS GENÉTICOS </center>

In [1]:
#nativos
import random

# terceros
import numpy as np
from difflib import SequenceMatcher as SM

In [2]:
objteivo = "chaymantaqa ripusaqmi, hanaymanpas uraymanpas, wayra hina muyurispay, mayu hina qaparispay."

valores_posibles = list(range(97, 123))
print(valores_posibles)

[97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122]


### ACOPLAMIENTO DE VALORES POSIBLES

In [3]:
def validacion_valores_posible(objteivo, valores_posibles):
    valores_posibles += list({ord(caracter) for caracter in objteivo if ord(caracter)})
    
    return list(set(valores_posibles))

valores_posibles = validacion_valores_posible(objteivo, valores_posibles)
print(valores_posibles)

[32, 44, 46, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122]


### CREACION DE PRIMERA POBLACION

In [4]:
def genera_poblacion_inicial_aleatoria(objteivo, n_poblacion, valores_posibles):
    return [
        ''.join(
            [chr(random.choice(valores_posibles)) for _ in range(len(objteivo))]
        ) for veces in range(n_poblacion)
    ]

### FUNCION DE COMPARACION CON OBJETIVO

In [5]:
primer_corte = int(1*len(objteivo) / 4)
segundo_corte = int(2*len(objteivo) / 4)
tercer_corte = int(3*len(objteivo) / 4)

primer_corte, segundo_corte, tercer_corte

(22, 45, 68)

In [6]:
def fitness_directo(individuo, objteivo):
    # comparacion en tres puntos
    return SM(None, objteivo, individuo).ratio()


def fitness_tres_cortes(individuo, objteivo):
    # comparacion en tres puntos
    
    valores = np.array([
        SM(None, objteivo[:primer_corte], individuo[:primer_corte]).ratio(),
        SM(None, objteivo[primer_corte:segundo_corte], individuo[primer_corte:segundo_corte]).ratio(),
        SM(None, objteivo[segundo_corte:tercer_corte], individuo[segundo_corte:tercer_corte]).ratio(),
        SM(None, objteivo[tercer_corte:], individuo[tercer_corte:]).ratio()
    ])
    #print(valores)
    return np.median(valores)

def fitness_conteo(individuo, objteivo):
    n_objteivo = len(objteivo)
    coincidentes = [1 for i in range(n_objteivo) if individuo[i] == objteivo[i]]
    
    return sum(coincidentes) / n_objteivo

### FUNCION DE CRUZA


In [7]:
def cruza_tres_cortes(individuo1, individuo2):
    return individuo1[:primer_corte] + \
           individuo2[primer_corte:segundo_corte] + \
           individuo1[segundo_corte:tercer_corte] + \
           individuo2[tercer_corte:]

def cruza_un_corte_al_azar(individuo1, individuo2):
    corte_al_azar = random.randint(0, len(objteivo) - 1)
    
    return individuo1[:corte_al_azar] + \
           individuo2[corte_al_azar:]

### FUNCION DE MUTACION

In [8]:
def mutacion(individuo3, valores_posibles, ratio_mutacion):
    individuo3_list = [caracter for caracter in individuo3]
    
    for posicion, caracter in enumerate(individuo3_list):
        # A mayor indice_de_mutacion no se realizará la mmutacion
        indice_de_mutacion = random.random()
        
        if indice_de_mutacion <= ratio_mutacion:
            individuo3_list[posicion] = chr(random.choice(valores_posibles))
    
    return ''.join(individuo3_list)

### AG

In [9]:
print(valores_posibles)

[32, 44, 46, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122]


In [15]:
n_poblacion = 2000

poblacion_inicial = genera_poblacion_inicial_aleatoria(objteivo, n_poblacion, valores_posibles)
poblacion_inicial[:3]

['tthkllim,.wbeu,ijky..jvyfsrsvkjdvdmqqzbpfhrowqh,.jqwaxwvtaumhsnfjqd.cyd,yc.lbzoiunzindmnz i',
 'iewqjb,rzdnhwf.tr..rorjffu.qvrjazul qgiwb.cqqvez bbqak ztgwunefmeqesplq.ljavdecwknkoc.ecjri',
 'vkguzlrozrr ,kiu,drnuw  xmhgumnlihwrxjk.bablfcshfyefajclgoijynogcmwmqrkupzzgwqukhlasf.yqsev']

In [21]:
objteivo = "chaymantaqa ripusaqmi, hanaymanpas uraymanpas, wayra hina muyurispay, mayu hina qaparispay."
ratio_mutacion = 0.05

In [24]:
poblacion = poblacion_inicial
generacion = 1

while True:
    hallado = False

    # EVALUACION
    for indice, individuo in enumerate(poblacion):
        fitness = fitness_conteo(individuo, objteivo)
        #print('generacion [{}], individuo orden [{}] fitness [{}]:  {}'.format(generacion, indice, round(fitness, 3), individuo))

        if fitness == 1:
            print('generacion [{}], individuo orden [{}] fitness [{}]:  {}'.format(generacion, indice, round(fitness, 3), individuo))
            hallado = True
            break


    if hallado:
        break

    # SELECCION
    siguiente_poblacion = []
    n_seleccionados = int(n_poblacion*10/100)

    # Obtengo el score por partes para seleccionar el 10% mejor
    individuos_score_partes = sorted([
            (individuo, fitness_conteo(individuo, objteivo)) for individuo in poblacion
        ], 
        key=lambda _:_[1], 
        reverse=True
    )[:n_seleccionados]
    print('generacion [{}], mejor score [{}]'.format(generacion, individuos_score_partes[0]))
    
    # Seleccionados con la cantidad de veces a reproducirse
    padres_seleccionados = dict([(_[0], int(round(_[1]*100))) for _ in individuos_score_partes])
    #print(padres_seleccionados)
    
    #selecccionados con chance de reproducirse
    seleccionados_con_chance = [k for k, v in padres_seleccionados.items() if v > 0]
    
    n_con_chance = len(seleccionados_con_chance)
    #print(n_con_chance)
    n_reproduccion = 0
    
    # REPRODUCCION
    while n_con_chance not in [0, 1] and n_reproduccion < n_poblacion*3:
        #print("->", n_con_chance, n_reproduccion, n_poblacion)
        orden_azar_1 = random.randint(0, n_con_chance - 1)
        orden_azar_2 = random.randint(0, n_con_chance - 1) 
        
        if orden_azar_1 == orden_azar_2:
            #print("orden repetido - no reproduccion")
            continue
            
        padre_a = seleccionados_con_chance[orden_azar_1]
        padre_b = seleccionados_con_chance[orden_azar_2]
        
        if padre_a == padre_b:
            #print("mismos padres - no ]reproduccion")
            continue

        # Aplico cruza
        hijo = cruza_un_corte_al_azar(padre_a, padre_b)

        # Aplico mutacion
        mutado = mutacion(hijo, valores_posibles, ratio_mutacion)
        siguiente_poblacion.append(mutado)

        #Los padres disminuyen la chance de volver a reporducirse
        padres_seleccionados[padre_a] -= 1
        padres_seleccionados[padre_b] -= 1

        # reinicio valores
        seleccionados_con_chance = [k for k, v in padres_seleccionados.items() if v > 0]
        n_con_chance = len(seleccionados_con_chance)
        n_reproduccion += 1



    generacion += 1
    # la siguiente generacion seran los hijos 
    poblacion = siguiente_poblacion + [_[0] for _ in individuos_score_partes[:int(n_seleccionados/2)]]
    #print("/", len(poblacion))
    
    if generacion == 150:
        break    


generacion [1], mejor score [('jzpjeyk,nqffr,fyf.x.jsglbzhh urztazjhxicayhauom cyxy.wbipmmltbwysfhyt ntoi xkc ,q,.yckztf a', 0.12087912087912088)]
generacion [2], mejor score [('ebah,dtkqumaxprbfufsuk wggjycgotrdq,dayknybtp urjy,uthqnuxjbeqdmjxmy.wueru iqzvmofpsuh,mudx', 0.13186813186813187)]
generacion [3], mejor score [('blntgariallirzv rzqdrbphggjocgotrdq,dayknybtp urjy,uthqnuxjbeqdmjxmy.wueru iqzvmofpsuu,mudx', 0.15384615384615385)]
generacion [4], mejor score [('vpzsomallhpfnrprs,qcr,qimzrkvhcajfxncpjxaqsnkg dvg,jmtze,m,yyprrspk,dmmahxbfbnlqqdparbhbzvs', 0.18681318681318682)]
generacion [5], mejor score [('rhwwd.niwzgiayouw,wmgascan hfrcmfsihr iegn au,u,njbgfbisnwlkdgdmbxmy.iuerr fnnlqqdparrhbzvs', 0.2087912087912088)]
generacion [6], mejor score [('zhhvmaqtrihg.sorsqenb,cwo ,vmtornwcnralbvnbts.sgscwpdhsnvm,gyprrslqmdsmayxbzbnlqqdparwhbzvs', 0.26373626373626374)]
generacion [7], mejor score [('cgajuangnqjxzi,opwmf,,,vmoachmbp,qbjvalmanptsmobv,fz pnndm,yyprpppk,domahubfbnlqqd.arbhb